In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from scipy.cluster import hierarchy
from collections import defaultdict
from scipy.spatial.distance import squareform
import xgboost as xgb
import shap
import scipy.stats
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_path = "your_data.csv"
df = pd.read_csv(data_path)

id_col = "Participant ID"
label_col = "status"
group_col = "group"
feature_cols = [c for c in df.columns if c not in [id_col, label_col, group_col]]
df_train = df[df[group_col] == "train"]
df_test = df[df[group_col] == "test"]       # internal validation

X = df[feature_cols].copy()
y = df[label_col].copy()
X_train = df_train[feature_cols].copy()
y_train = df_train[label_col].copy()
X_test = df_test[feature_cols].copy()
y_test = df_test[label_col].copy()
print(df_train.shape)
print(df_test.shape)
print("Number of features:", len(feature_cols))

In [ ]:
correlation_matrix = X.corr(method='spearman')
# correlation_matrix.to_csv("correlation_matrix.csv")

distance_matrix = 1 - np.abs(correlation_matrix)
dist_array = squareform(distance_matrix)
dist_linkage = hierarchy.linkage(dist_array, method='ward')

fig, ax = plt.subplots(figsize=(30, 16))
dendro = hierarchy.dendrogram(dist_linkage, labels=correlation_matrix.columns, ax=ax)
ax.set_xticklabels(dendro["ivl"], rotation=60, fontsize=4, horizontalalignment='right')
ax.axhline(y=0.5, color='r', linewidth=2, linestyle='--')
plt.show()

cluster_ids = hierarchy.fcluster(dist_linkage, 0.5, criterion="distance")
feature_clus = pd.DataFrame({
    'Feature': feature_cols,
    'ClusterID': cluster_ids
})
# feature_clus.to_csv("Stroke_feature_clusters.csv", index=False)

In [ ]:
df_feature = X_train.copy()
df_group = y_train.copy()
scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
params = {
    'alpha': 1.5,
    'lambda': 1.5,
    'colsample_bytree': 0.75,
    'n_estimators': 400,
    'learning_rate': 0.01,
    'max_depth': 3,
    'subsample': 0.7,
    'min_child_weight': 3,
    'gamma': 0.9,
    'eval_metric':'auc',
    'scale_pos_weight': scale_pos_weight
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
def calculate_auc_for_feature(RNA_feature, df_feature, df_group, cv, params):
    aucs = []

    for train_idx, test_idx in cv.split(df_feature, df_group):
        X_train, X_test = df_feature.iloc[train_idx], df_feature.iloc[test_idx]
        y_train, y_test = df_group.iloc[train_idx], df_group.iloc[test_idx]

    model = xgb.XGBClassifier(**params)
    model.fit(X_train[[RNA_feature]], y_train)

    y_pred_prob = model.predict_proba(X_test[[RNA_feature]])[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)
    aucs.append(auc)
    return np.mean(aucs)

auc_scores = {}
for RNA_feature in df_feature.columns:
    auc_score = calculate_auc_for_feature(RNA_feature, df_feature, df_group, cv, params)
    auc_scores[RNA_feature] = auc_score

feature_auc = pd.DataFrame(list(auc_scores.items()), columns=['Feature', 'AUC'])
# feature_auc.to_csv("RNA_auc_scores.csv", index=False)
feature_auc.head()

In [ ]:
feature_sel = pd.merge(feature_auc, feature_clus, how='left', on="Feature")
best_features = feature_sel.loc[feature_sel.groupby('ClusterID')['AUC'].idxmax()]
# best_features.to_csv('RNA_best_features.csv', index=False)

In [ ]:
selected = best_features["Feature"].tolist()
X_train2 = X_train[selected].copy()
X_test2 = X_test[selected].copy()

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    'alpha': [1.5],
    'lambda': [1.5],
    'colsample_bytree': [0.75],
    'n_estimators': [400],
    'learning_rate': [0.01],
    'max_depth': [3],
    'subsample': [0.7],
    'min_child_weight': [3],
    'gamma': [0.9],
    'scale_pos_weight': [scale_pos_weight]
}

grid_search = GridSearchCV(
    estimator=xgb.XGBClassifier(
        random_state=42,
        scale_pos_weight=scale_pos_weight,
        eval_metric='auc',
    ),
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc',
    n_jobs=4,
    verbose=1
)

grid_search.fit(X_train2, y_train)
best_model = grid_search.best_estimator_

In [ ]:
explainer = shap.TreeExplainer(best_model)
batch_size = 100
shap_values_list = []

for i in range(0, len(X_test2), batch_size):
    batch = X_test2.iloc[i:i+batch_size]
    shap_values_batch = explainer.shap_values(batch)
    if isinstance(shap_values_batch, list):
        shap_values_batch = shap_values_batch[1]
    shap_values_list.append(shap_values_batch)
    print(f"Processed batch {i//batch_size + 1}/{(len(X_test2)//batch_size)+1}")

shap_values = np.concatenate(shap_values_list)
# pd.DataFrame(shap_values, columns=X_test2.columns).to_csv('allrna_xgboost_shap_values.csv', index=False)

plt.figure()
shap.summary_plot(shap_values, X_test2, max_display=20, show=False)
plt.show()

In [ ]:
params = {
    'alpha': 1.5,
    'lambda': 1.5,
    'colsample_bytree': 0.75,
    'n_estimators': 400,
    'learning_rate': 0.01,
    'max_depth': 3,
    'subsample': 0.7,
    'min_child_weight': 3,
    'gamma': 0.9,
    'eval_metric':'auc',
    'scale_pos_weight': scale_pos_weight,
    'random_state': 42
}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

def normal_imp(mydict):
    mysum = sum(mydict.values())
    for key in mydict.keys():
        mydict[key] = mydict[key] / mysum
    return mydict

tg_imp_cv = Counter()
shap_imp_cv = np.zeros(X_train2.shape[1])

for train_idx, test_idx in cv.split(X_train2, df_group):
    X_train, X_test = X_train2.iloc[train_idx, :], X_train2.iloc[test_idx, :]
    y_train, y_test = df_group.iloc[train_idx], df_group.iloc[test_idx]

    my_xgb = XGBClassifier(**params)
    my_xgb.fit(X_train, y_train)

    totalgain_imp = my_xgb.feature_importances_
    totalgain_imp = dict(zip(X_train2.columns, totalgain_imp.tolist()))

    tg_imp_cv += Counter(normal_imp(totalgain_imp))

    explainer = shap.TreeExplainer(my_xgb)
    shap_values = explainer.shap_values(X_test)
    shap_values_mean = np.mean(np.abs(shap_values), axis=0)
    shap_imp_cv += shap_values_mean / np.sum(shap_values_mean)

In [ ]:
shap_imp_df = pd.DataFrame({
    'Analytes': X_train2.columns,
    'ShapValues_cv': shap_imp_cv / 10
})
shap_imp_df.sort_values(by='ShapValues_cv', ascending=False, inplace=True)

stats_summary = {
    'Mean': shap_imp_df['ShapValues_cv'].mean(),
    'Std': shap_imp_df['ShapValues_cv'].std(),
    'Min': shap_imp_df['ShapValues_cv'].min(),
    'Max': shap_imp_df['ShapValues_cv'].max(),
    '25%': shap_imp_df['ShapValues_cv'].quantile(0.25),
    '50% (Median)': shap_imp_df['ShapValues_cv'].median(),
    '75%': shap_imp_df['ShapValues_cv'].quantile(0.75)
}

print("SHAP Values CV Statistics:")
for stat, value in stats_summary.items():
    print(f"{stat}: {value:.4f}")

In [ ]:
tg_imp_cv = normal_imp(tg_imp_cv)
tg_imp_df = pd.DataFrame({
    'Analytes': list(tg_imp_cv.keys()),
    'TotalGain_cv': list(tg_imp_cv.values())
})
tg_imp_df

In [ ]:
my_imp_df = pd.merge(left=shap_imp_df, right=tg_imp_df, how='left', on=['Analytes'])
my_imp_df['Ensemble_cv'] = (my_imp_df['ShapValues_cv'] + my_imp_df['TotalGain_cv']) / 2
my_imp_df.sort_values(by='TotalGain_cv', ascending=False, inplace=True)
print('finished')

In [ ]:
def get_imp_analy(my_imp_df, top_prop=0.9):
    imp_score, iter = 0, 0
    while imp_score < top_prop and iter < len(my_imp_df):
        imp_score += my_imp_df.Ensemble_cv.iloc[iter]
        iter += 1
    return iter

top_feature_count = get_imp_analy(my_imp_df, top_prop=0.9)

print(f"Number of proteins contributing to over 90% of overall information gains: {top_feature_count}")

top_features_df = my_imp_df.iloc[:top_feature_count]
# top_features_df.to_csv('Top_InfoGain_features.csv', index=False)
# print('Top-ranked proteins saved to CSV.')

In [ ]:
top_features = top_features_df['Analytes'].tolist()
X_train3 = X_train[top_features]
X_test3 = X_test[top_features]

In [ ]:
# AUC comparison adapted from
def compute_midrank(x):
    """Computes midranks."""
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5 * (i + j - 1)
        i = j
    T2 = np.empty(N, dtype=float)
    T2[J] = T + 1
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """
    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }
    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in range(k):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, covar):
    """Computes log(10) of p-values.
    Args:
       aucs: 1D array of AUCs
       covar: AUC DeLong covariances
    Returns:
       log10(pvalue)
    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, covar), l.T))  # 将 sigma 改为 covar
    return np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)


def compute_ground_truth_statistics(ground_truth):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    return order, label_1_count


def delong_roc_variance(ground_truth, predictions):
    """
    Computes ROC AUC variance for a single set of predictions
    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """
    Computes log(p-value) for hypothesis that two ROC AUCs are different
    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1
    """
    order, label_1_count = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)

params = {
    'alpha': 1.5,
    'lambda': 1.5,
    'colsample_bytree': 0.75,
    'n_estimators': 400,
    'learning_rate': 0.01,
    'max_depth': 3,
    'subsample': 0.7,
    'min_child_weight': 3,
    'gamma': 0.9,
    'eval_metric':'auc',
    'scale_pos_weight': scale_pos_weight
}
xgb_model = xgb.XGBClassifier(**params)

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

y_pred_lst_prev1 = np.zeros(len(df_group))
y_pred_lst_prev2 = np.zeros(len(df_group))
y_pred_lst_prev3 = np.zeros(len(df_group))
tmp_f, AUC_cv_lst = [], []

for f in top_features:
    tmp_f.append(f)
    my_X = X_train3[tmp_f]

    AUC_cv, y_pred_lst, y_true_lst = [], [], []

    for train_idx, test_idx in cv.split(my_X, df_group):
        X_train, X_test = my_X.iloc[train_idx, :], my_X.iloc[test_idx, :]
        y_train, y_test = df_group.iloc[train_idx], df_group.iloc[test_idx]

        my_xgb = xgb.XGBClassifier(**params)
        my_xgb.fit(X_train, y_train)

        y_pred_prob = my_xgb.predict_proba(X_test)[:, 1]
        AUC_cv.append(roc_auc_score(y_test, y_pred_prob))

        y_pred_lst += y_pred_prob.tolist()
        y_true_lst += y_test.tolist()

    auc_full = roc_auc_score(y_true_lst, y_pred_lst)

    log10_p1 = delong_roc_test(np.array(y_true_lst), np.array(y_pred_lst_prev1), np.array(y_pred_lst))
    log10_p2 = delong_roc_test(np.array(y_true_lst), np.array(y_pred_lst_prev2), np.array(y_pred_lst))
    log10_p3 = delong_roc_test(np.array(y_true_lst), np.array(y_pred_lst_prev3), np.array(y_pred_lst))

    print(f"Feature: {f}, Delong p-values: {log10_p1}, {log10_p2}, {log10_p3}")

    y_pred_lst_prev3 = y_pred_lst_prev2
    y_pred_lst_prev2 = y_pred_lst_prev1
    y_pred_lst_prev1 = y_pred_lst


    tmp_out = np.array([np.mean(AUC_cv), np.std(AUC_cv), 10**log10_p1[0][0], 10**log10_p2[0][0], 10**log10_p3[0][0], auc_full])
    AUC_cv_lst.append(tmp_out)
    print(f"Feature: {f}, AUC Results: {tmp_out}")

In [ ]:
tmp_out = np.array([
    np.mean(AUC_cv),
    np.std(AUC_cv),
    10**log10_p1[0][0],
    10**log10_p2[0][0],
    10**log10_p3[0][0],
    auc_full
])
print(f"Feature: {f}, AUC Results: {tmp_out}")

AUC_df = pd.DataFrame(AUC_cv_lst, columns=['AUC_mean', 'AUC_std', 'Delong1', 'Delong2', 'Delong3', 'AUC_all'])
AUC_df[['AUC_mean', 'AUC_std', 'AUC_all']] = np.round(AUC_df[['AUC_mean', 'AUC_std', 'AUC_all']], 3)

AUC_df = pd.concat((pd.DataFrame({'Analytes': tmp_f}), AUC_df), axis=1)
AUC_df.to_csv('Delong_Selection_Results.csv', index=False)
print('Finished feature selection and saved results.')

In [ ]:
top_features_df.rename(columns = {'Ensemble_cv': 'sRNA_imp'}, inplace = True)
mydf = pd.merge(AUC_df, top_features_df, how = 'left', on = ['Analytes'])
mydf

In [ ]:
def get_nb_f(mydf):
    p_lst = mydf.Delong2.tolist()
    i = 0
    while((p_lst[i]<0.05)|(p_lst[i+1]<0.05)):
        i+=1
    return i

mydf['AUC_lower'] = mydf['AUC_mean'] - mydf['AUC_std']
mydf['AUC_upper'] = mydf['AUC_mean'] + mydf['AUC_std']
mydf['AUC_upper'].iloc[mydf['AUC_upper']>=1] = 1
mydf['rna_idx'] = [i for i in range(1, len(mydf)+1)]
nb_f = get_nb_f(mydf)

fig, ax = plt.subplots(figsize = (18, 6.5))
palette = sns.color_palette("Blues",n_colors=len(mydf))
palette.reverse()
sns.barplot(ax=ax, x = "Analytes", y = "sRNA_imp", palette=palette, data=mydf.sort_values(by="sRNA_imp", ascending=False))
y_imp_up_lim = round(mydf['sRNA_imp'].max() + 0.01, 2)
ax.set_ylim([0, y_imp_up_lim])
ax.tick_params(axis='y', labelsize=14)
ax.set_xticklabels(mydf['Analytes'], rotation=45, fontsize=10, horizontalalignment='right')
my_col = ['r']*nb_f + ['k']*(len(mydf)-nb_f)
for ticklabel, tickcolor in zip(plt.gca().get_xticklabels(), my_col):
    ticklabel.set_color(tickcolor)

ax.set_ylabel('sRNA importance', weight='bold', fontsize=18)
ax.set_xlabel('')
ax.grid(which='minor', alpha=0.2, linestyle=':')
ax.grid(which='major', alpha=0.5,  linestyle='--')
ax.set_axisbelow(True)

ax2 = ax.twinx()
ax2.plot(np.arange(nb_f+1), mydf['AUC_mean'][:nb_f+1], 'red', alpha = 0.8, marker='o')
ax2.plot(np.arange(nb_f+1, len(mydf)), mydf['AUC_mean'][nb_f+1:], 'black', alpha = 0.8, marker='o')
ax2.plot([nb_f, nb_f+1], mydf['AUC_mean'][nb_f:nb_f+2], 'black', alpha = 0.8, marker='o')
plt.fill_between(mydf['rna_idx']-1, mydf['AUC_lower'], mydf['AUC_upper'], color = 'tomato', alpha = 0.2)
ax2.set_ylabel('Cumulative AUC', weight='bold', fontsize=18)
ax2.tick_params(axis='y', labelsize=14)
y_auc_up_lim = round(mydf['AUC_upper'].max() + 0.01, 2)
y_auc_low_lim = round(mydf['AUC_lower'].min() - 0.01, 2)
ax2.set_ylim([y_auc_low_lim, y_auc_up_lim])

fig.tight_layout()
plt.xlim([-.6, len(mydf)-.2])
plt.show()